In [18]:
import logging
import random

import numpy as np
import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')

# set seed
seed = 666
random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

In [19]:
# split data to 10 fold
fold_num = 10
data_file = 'zhengti_fenci.csv'
import pandas as pd


def all_data2fold(fold_num):
    fold_data = []
    f = pd.read_csv(data_file, encoding='UTF-8')
    texts = f['text'].tolist()
    labels = f['label'].tolist()

    total = len(labels)

    index = list(range(total))
    np.random.shuffle(index)

    all_texts = []
    all_labels = []
    for i in index:
        all_texts.append(texts[i])
        all_labels.append(labels[i])

    label2id = {}
    for i in range(total):
        label = str(all_labels[i])
        if label not in label2id:
            label2id[label] = [i]
        else:
            label2id[label].append(i)

    all_index = [[] for _ in range(fold_num)]
    for label, data in label2id.items():
        # print(label, len(data))
        batch_size = int(len(data) / fold_num)
        other = len(data) - batch_size * fold_num
        for i in range(fold_num):
            cur_batch_size = batch_size + 1 if i < other else batch_size
            # print(cur_batch_size)
            batch_data = [data[i * batch_size + b] for b in range(cur_batch_size)]
            all_index[i].extend(batch_data)

    batch_size = int(total / fold_num)
    other_texts = []
    other_labels = []
    other_num = 0
    start = 0
    for fold in range(fold_num):
        num = len(all_index[fold])
        texts = [all_texts[i] for i in all_index[fold]]
        labels = [all_labels[i] for i in all_index[fold]]

        if num > batch_size:
            fold_texts = texts[:batch_size]
            other_texts.extend(texts[batch_size:])
            fold_labels = labels[:batch_size]
            other_labels.extend(labels[batch_size:])
            other_num += num - batch_size
        elif num < batch_size:
            end = start + batch_size - num
            fold_texts = texts + other_texts[start: end]
            fold_labels = labels + other_labels[start: end]
            start = end
        else:
            fold_texts = texts
            fold_labels = labels

        assert batch_size == len(fold_labels)

        # shuffle
        index = list(range(batch_size))
        np.random.shuffle(index)

        shuffle_fold_texts = []
        shuffle_fold_labels = []
        for i in index:
            shuffle_fold_texts.append(fold_texts[i])
            shuffle_fold_labels.append(fold_labels[i])

        data = {'label': shuffle_fold_labels, 'text': shuffle_fold_texts}
        fold_data.append(data)

    logging.info("Fold lens %s", str([len(data['label']) for data in fold_data]))

    return fold_data


fold_data = all_data2fold(10)


2022-06-01 11:22:55,163 INFO: Fold lens [1091, 1091, 1091, 1091, 1091, 1091, 1091, 1091, 1091, 1091]


AttributeError: 'dict' object has no attribute 'to_csv'

In [20]:
# build train data for word2vec
# 按照训练集来构建word2vec,所以是8份数据
fold_id = 8

train_texts = []
for i in range(0, fold_id):
    data = fold_data[i]
    train_texts.extend(data['text'])

logging.info('Total %d docs.' % len(train_texts))

2022-06-01 11:22:55,192 INFO: Total 9819 docs.


In [21]:
logging.info('Start training...')
from gensim.models.word2vec import Word2Vec

num_features = 300     # Word vector dimensionality
num_workers = 8       # Number of threads to run in parallel

train_texts = list(map(lambda x: list(x.split()), train_texts))

# 采用skip-gram，负采样，训练10轮，12核CPU大约需要4个小时
model = Word2Vec(train_texts, sg=1, workers=num_workers, vector_size=num_features, compute_loss=True,
                 epochs=10, hs=0, window=10)

model.init_sims(replace=True)

# save model
model.save("./word2vec.bin")


2022-06-01 11:22:55,246 INFO: Start training...
2022-06-01 11:22:55,746 INFO: collecting all words and their counts
2022-06-01 11:22:55,747 INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-06-01 11:22:56,497 INFO: collected 84615 word types from a corpus of 2302557 raw words and 9819 sentences
2022-06-01 11:22:56,498 INFO: Creating a fresh vocabulary
2022-06-01 11:22:56,715 INFO: Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 22549 unique words (26.65% of original 84615, drops 62066)', 'datetime': '2022-06-01T11:22:56.675171', 'gensim': '4.2.0', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'prepare_vocab'}
2022-06-01 11:22:56,716 INFO: Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 2202489 word corpus (95.65% of original 2302557, drops 100068)', 'datetime': '2022-06-01T11:22:56.716169', 'gensim': '4.2.0', 'python': '3.8.13 (default, Mar 28 2

KeyboardInterrupt: 

In [ ]:
# load model
model = Word2Vec.load("./word2vec.bin")

# convert format
model.wv.save_word2vec_format('./word2vec.txt', binary=False)
